In [1]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
from sklearn import datasets

import core.container
from core.container import OperationContainer
from core.record_event import RecordEvent
from core.record_event import ClearEvent

from sklearn.neighbors import KNeighborsClassifier

ClearEvent()

In [2]:
@RecordEvent
def cell_2():
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    
    return OperationContainer({"iris_X": iris_X, "iris_y": iris_y})

In [3]:
iris_X, iris_y = cell_2()


rtv =  <core.container.OperationContainer object at 0x000001D4FBD59E70>
key =  iris_X 
Item =  [[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1

In [ ]:
@RecordEvent
def cell_3():
    np.random.seed(0)

In [ ]:
cell_3()

In [ ]:
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return OperationContainer({"indices": indices, "iris_X_train": iris_X_train, "iris_y_train": iris_y_train, "iris_X_test": iris_X_test, "iris_y_test": iris_y_test})

In [ ]:
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()

In [ ]:
@RecordEvent
def cell_5():
    knn = KNeighborsClassifier()
    knn.fit(iris_X_train, iris_y_train)

    knn.predict(iris_X_test)
    
    return OperationContainer({"knn": knn})

In [ ]:
knn = cell_5()

In [ ]:
for x in core.event.operation_events:
    print(x, "\n")
    print("related data events: \n")
    for y in x.related_data_events:
        print(y, "\n")
    print("------------")

In [ ]:
for x in core.event.data_events:
   print(x, "\n")

In [ ]:
for x in core.event.data_containers:
    print(x, "\n")

In [ ]:
parent_lookup = {oe: set() for oe in core.event.operation_events}
children_lookup = {oe: set() for oe in core.event.operation_events}

In [ ]:
print(core.event.operation_event_lookup)

In [ ]:
for oe in core.event.operation_events:
    for related_data_event in core.event.operation_event_lookup[oe.exec_uuid].related_data_events:
        parent_oe = related_data_event.prev_operation_event
        
        parent_lookup[oe].add(parent_oe)
        children_lookup[parent_oe].add(oe)

In [ ]:
print(parent_lookup)
print(children_lookup)

In [ ]:
from core.convert_graph import ConvertGraph
graph = ConvertGraph()

In [ ]:
print(graph.active_nodes)
for edge in graph.edges:
    print(edge.oe.cell_func_name, edge.duration)
    print("sources")
    for node in edge.src.nodes:
        print(node.var.name, node.var.ver, node.size)
    print("destinations")
    for node in edge.dst.nodes:
        print(node.var.name, node.var.ver, node.size)

In [ ]:
from algorithm.optimizer_greedy import OptimizerGreedy

graph.set_migration_speed(100000)
opt = OptimizerGreedy(graph, list(graph.active_nodes.values()))
recompute_nodes = opt.select_nodes()

for node in recompute_nodes:
    print(node.var.name, node.var.ver, node.size)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

#nx.draw(opt.compute_graph, with_labels = True)
#plt.show()

print(opt.active_node_sets)
print(opt.recomputation_edges)

recompute_idx = set()
recompute_idx.add(1)

migrate_idx = set()
migrate_idx.add(5)
migrate_idx.add(7)

print(opt.compute_migration_cost(migrate_idx))
print(opt.compute_recomputation_cost(recompute_idx))
print(opt.total_cost(migrate_idx))

In [ ]:
from core.graph.search import find_path

recompute_edges = find_path(graph, recompute_nodes, core.event.operation_events)
for edge in recompute_edges:
    print(edge.oe.cell_func_name)
    print(edge.oe.cell_func_code)

In [ ]:
from core.io.migrate import migrate
print(recompute_nodes, recompute_edges)
recomputation_code = migrate(recompute_nodes, recompute_edges, "numpy_notebook")
for code in recomputation_code:
    print("---------------------------------------")
    print(code)

In [ ]:
import sys
mods = [m.__name__ for m in sys.modules.values() if m]
print(mods)